In [ ]:
import json
from pathlib import Path

import pandas

In [ ]:
with open("../config.json", "r") as fh:
    config = json.load(fh)
base_path = Path(config["base_path"])
base_path

In [ ]:
with open("../scratch/polder_site_relations.json", 'r') as fh:
    polder_site_relations = json.load(fh)

In [ ]:
summary = pandas.read_csv("../scratch/polder_rp_damages.csv")